In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import desc
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import Column, Integer, String, Float, Date

In [ ]:
# Creates an engien and a connection
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect= True)

In [ ]:
Base.classes.keys()

In [ ]:
inspector =inspect(engine)

inspector.get_table_names()

In [ ]:
columns = inspector.get_columns("measurement")
for col in columns:
    print(col["name"], col["type"])

In [ ]:
# We can view all of the classes that automap found
measure_df = pd.read_sql("SELECT * FROM measurement", conn)
measure_df.head()

In [ ]:
station_df = pd.read_sql("SELECT * FROM station", conn)
station_df.head()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Stations = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Retreives a count. (I wanted to see how many dates I was working with)
year_2017 = session.query(Measurement).group_by(Measurement.date).count()
year_2017

In [ ]:
precip_df = pd.DataFrame(prcp_date)
precip_df

In [ ]:
# Gets the msot recetn date 
recent_date = session.query(Measurement.date).order_by(desc(Measurement.date)).first()
#recent_date


# Calculate the date 1 year ago from the last data point in the database
a_year_ago = session.query(Measurement.date).filter((Measurement.date) == dt.date(2016,8,23)).first()


# Design a query to retrieve the last 12 months of precipitation data and plot the results
last_12 = session.query(Measurement.date).filter((Measurement.date) >= dt.date(2016,8,23)).all()


# Perform a query to retrieve the data and precipitation scores
prcp_date = session.query(Measurement.date, Measurement.prcp).filter((Measurement.date) >= dt.date(2016,8,23)).all()


# Save the query results as a Pandas DataFrame and set the index to the date column
precip_df = pd.DataFrame(prcp_date)
#precip_df.head()
precip_df= precip_df.sort_values('date')

# Sort the dataframe by date
precip_df.set_index("date", inplace= True)



In [ ]:
precip_df.plot.bar()
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

fig, ax = plt.subplots(figsize = (20, 5))
precip_df.plot(ax = ax, x_compat = True)

# creates title and labels
ax.set_xlabel('Date')
ax.set_ylabel('Inches')
ax.set_title("A Year of Rainy Days")

# Saves image 
plt.savefig("Images/rainydays.png")

# Shwocases the graph
plt.tight_layout()
plt.show()



In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precip_df.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?
station_amount = session.query(Measurement).group_by(Measurement.station).count()
station_amount

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
station_amount_count = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
                       
station_amount_count

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
most_popular_station = 'USC00519281'

temp = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.station == most_popular_station).all()
temp

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
station_observation = session.query(Measurement.date, Measurement.tobs).filter(Measurement.station == most_popular_station).filter((Measurement.date) >= dt.date(2016, 8, 23)).all()

#save as a data frame
station_observation_df = pd.DataFrame(station_observation, columns = ['date', 'temperature'])

#Creating the base fo the graph
fig, ax = plt.subplots()
station_observation_df.plot.hist(bins = 12, ax = ax)

# set labels
ax.set_xlabel('Temperature')
ax.set_ylabel('Frequency')

#save figure
plt.savefig("Images/yearly_plot.png")

#plot
plt.tight_layout()
plt.show()

## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
